In [1]:
import d4rl
import gym
import numpy as np

/home/users/adarsh/anaconda3/envs/diffuser/lib/python3.8/site-packages/glfw/__init__.py:916: GLFWError: (65544) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)
pybullet build time: May 20 2022 19:44:17


In [2]:
env = gym.make('maze2d-umaze-v1')
dataset = env.get_dataset()
np.where(dataset['timeouts']==1)

load datafile: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 30.92it/s]


(array([    91,    144,    230, ..., 999592, 999702, 999868]),)

In [ ]:
env.unwrapped.goal_locations

In [ ]:
env.state_vector()

In [ ]:
env.unwrapped.max_episode_steps = env._max_episode_steps
env.unwrapped.max_episode_steps

In [ ]:
{k:v[0:10] for k,v in dataset.items()}

In [ ]:
{k:v[85:110] for k,v in dataset.items()}

In [ ]:
n=15
print(dataset.keys())
print(dataset['actions'][n])
print(dataset['observations'][n])
print(dataset['infos/qpos'][n])
print(dataset['infos/qvel'][n])

In [ ]:
np.where(dataset['timeouts']==1)[0]

In [ ]:
{k:v[0:93] for k,v in dataset_naive.items()}

In [ ]:
import copy
from math import atan2, sin, cos, radians
dataset_naive = copy.deepcopy(dataset)
    

def calc_action(start, goal):
    delta_x = goal[0] - start[0]
    delta_y = goal[1] - start[1]
    theta_radians = atan2(delta_y, delta_x)
    a_x = cos(theta_radians)
    a_y = sin(theta_radians)
    return [a_x,a_y]
    

start = 0
for t in np.where(dataset['timeouts']==1)[0]:
    dataset_naive['actions'][start:t+1] = calc_action(dataset['infos/qpos'][start], dataset['infos/goal'][t])
    start = t+1
    
    

In [ ]:
dataset_naive['actions']

In [ ]:
dims = [1, 2, 4, 8]
list(zip(dims[:-1], dims[1:]))

In [ ]:
dims[1:]

In [8]:
import gym
import logging
from d4rl.pointmaze import waypoint_controller
from d4rl.pointmaze import maze_model
import numpy as np
import pickle
import gzip
import h5py
import argparse


def reset_data():
    return {'observations': [],
            'actions': [],
            'terminals': [],
            'rewards': [],
            'infos/goal': [],
            'infos/qpos': [],
            'infos/qvel': [],
            }

def append_data(data, s, a, tgt, done, env_data):
    data['observations'].append(s)
    data['actions'].append(a)
    data['rewards'].append(0.0)
    data['terminals'].append(done)
    data['infos/goal'].append(tgt)
    data['infos/qpos'].append(env_data.qpos.ravel().copy())
    data['infos/qvel'].append(env_data.qvel.ravel().copy())

def npify(data):
    for k in data:
        if k == 'terminals':
            dtype = np.bool_
        else:
            dtype = np.float32

        data[k] = np.array(data[k], dtype=dtype)

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('--render', action='store_true', help='Render trajectories')
    parser.add_argument('--noisy', action='store_true', help='Noisy actions')
    parser.add_argument('--env_name', type=str, default='maze2d-umaze-v1', help='Maze type')
    parser.add_argument('--num_samples', type=int, default=int(1e6), help='Num samples to collect')
    args = parser.parse_args()

    env = gym.make(args.env_name)
    maze = env.str_maze_spec
    max_episode_steps = env._max_episode_steps

    controller = waypoint_controller.WaypointController(maze)
    env = maze_model.MazeEnv(maze)

    env.set_target()
    s = env.reset()
    act = env.action_space.sample()
    done = False

    data = reset_data()
    ts = 0
    for _ in range(args.num_samples):
        position = s[0:2]
        velocity = s[2:4]
        act, done = controller.get_action(position, velocity, env._target)
        if args.noisy:
            act = act + np.random.randn(*act.shape)*0.5

        act = np.clip(act, -1.0, 1.0)
        if ts >= max_episode_steps:
            done = True
        append_data(data, s, act, env._target, done, env.sim.data)

        ns, _, _, _ = env.step(act)

        if len(data['observations']) % 10000 == 0:
            print(len(data['observations']))

        ts += 1
        if done:
            env.set_target()
            done = False
            ts = 0
        else:
            s = ns

        if args.render:
            env.render()

    
    if args.noisy:
        fname = '%s-noisy.hdf5' % args.env_name
    else:
        fname = '%s.hdf5' % args.env_name
    dataset = h5py.File(fname, 'w')
    npify(data)
    for k in data:
        dataset.create_dataset(k, data=data[k], compression='gzip')

In [17]:
env = gym.make('maze2d-open-v0')
maze = env.str_maze_spec
max_episode_steps = env._max_episode_steps

In [18]:
controller = waypoint_controller.WaypointController(maze)
env = maze_model.MazeEnv(maze)

In [20]:
env

In [55]:
def space_indices(num_steps, count):
    assert count <= num_steps

    if count <= 1:
        frac_stride = 1
    else:
        frac_stride = (num_steps - 1) / (count - 1)

    cur_idx = 0.0
    taken_steps = []
    for _ in range(count):
        taken_steps.append(round(cur_idx))
        cur_idx += frac_stride

    return taken_steps

In [64]:
steps = space_indices(100, 6)
log_count = min(len(steps) - 1, 10)
log_steps = [steps[i] for i in space_indices(len(steps) - 1, log_count)]
print(log_steps)
steps = steps[::-1]
pair_steps = zip(steps[:-1], steps[1:])

[0, 20, 40, 59, 79]


In [65]:
l = list(pair_steps)
l

[(99, 79), (79, 59), (59, 40), (40, 20), (20, 0)]